# Ring Appraisal Project

## URLs: 
### cartier_catalog.csv: https://www.kaggle.com/marcelopesse/cartier-jewelry-catalog
#### data set created April 2020

### jewelry.csv: https://www.kaggle.com/victormegir/jewelry-from-haritidisgr
#### data set created March 2021

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
import random as rand
import datetime as dt
import re

In [3]:
df_jewelry = pd.read_csv('jewelry.csv')
df_cartier = pd.read_csv('cartier_catalog.csv', usecols = ['categorie', 'title', 'price', 'tags', 'description'])

df_jewelry = df_jewelry.drop_duplicates().reset_index(drop=True)
df_cartier = df_cartier.drop_duplicates().reset_index(drop=True)

### Converting jewelry.csv price column to USD using 3/15/2021 exchange rate because of the scrape date.

#### Can be connected to an exchange rate API for live data.

In [4]:
df_jewelry['price_usd'] = round(df_jewelry['price'] * 1.19,2)

In [5]:
df_jewelry.head()

,name,price,sex,jewel_type,brand,material,color,jewel_weight,rocks,rock_details,dimensions,chain_carat,chain_length,diameter,price_usd
0,Set Necklace & Earrings Gloria Hope Silver / P...,75,Women,Necklaces,Gloria Hope,Metal Alloy,White,NaN,NaN,NaN,NaN,NaN,NaN,NaN,89.25
1,Set Necklace & Earrings Gloria Hope Silver / P...,109,Women,Necklaces,Gloria Hope,Metal Alloy,White,NaN,NaN,NaN,NaN,NaN,NaN,NaN,129.71
2,Set Necklace & Earrings Gloria Hope Silver / P...,99,Women,Necklaces,Gloria Hope,Metal Alloy,White,NaN,NaN,NaN,NaN,NaN,NaN,NaN,117.81
3,Set Necklace & Earrings Gloria Hope Silver,69,Women,Necklaces,Gloria Hope,Silver,White,NaN,NaN,NaN,NaN,NaN,NaN,NaN,82.11
4,Necklace tear Gloria Hope Silver,69,Women,Necklaces,Gloria Hope,Metal Alloy,White,NaN,NaN,NaN,Length 6cm,NaN,NaN,NaN,82.11


## Creating brand column in Cartier dataset

In [6]:
df_cartier['brand'] = 'Cartier'

In [7]:
df_cartier.head()

,categorie,title,price,tags,description,brand
0,rings,Love ring,1650.0,yellow gold,"Love ring, 18K yellow gold. Width: 5.5mm.",Cartier
1,rings,Love ring,3650.0,platinum,"Love ring, 950/1000 platinum. Width: 5.5mm.",Cartier
2,rings,Love ring,1650.0,pink gold,"Love ring, 18K rose gold. Width: 5.5mm.",Cartier
3,rings,Love ring,1770.0,white gold,"Love ring, 18K white gold. Width: 5.5mm.",Cartier
4,rings,Love wedding band,1070.0,yellow gold,"Love wedding band, 18K yellow gold. Width: 3.6mm.",Cartier


## How df_jewelry columns map to df_cartier:

### name = title, price_usd = price, jewel_type = categorie (kind of), other description columns = description (kind of) 

In [8]:
df_jewelry['color'].unique()

array(['White', 'Gold', 'Rose', nan, 'White, Gold & Rose', 'White & Gold',
       'White & Rose', 'Black', 'Gunmetal', 'Rose & Black', 'Gold & Rose'],
      dtype=object)

In [9]:
df_cartier['tags'].unique()

array(['yellow gold', 'platinum', 'pink gold', 'white gold',
       'yellow gold, diamonds', 'pink gold, diamonds',
       'white gold, diamonds', 'yellow gold, diamond',
       'pink gold, diamond', 'white gold, diamond',
       'white gold, ceramic, diamonds', 'pink gold, sapphire',
       'pink gold, sapphires, garnets, amethyst',
       'pink gold, white gold, diamonds',
       'pink gold, diamonds, emeralds, onyx',
       'yellow gold, tsavorite garnets, onyx',
       'white gold, emeralds, onyx, diamonds',
       'platinum, sapphires, emeralds, diamonds',
       'pink gold, tsavorite garnets, onyx',
       'yellow gold, onyx, tsavorite garnet',
       'white gold, tsavorite garnets, onyx',
       'yellow gold, lacquer, diamonds, tsavorite garnet',
       'pink gold, tsavorite garnets, onyx, black lacquer, diamonds',
       'pink gold, onyx, emeralds, diamonds',
       'yellow gold, emeralds, onyx, diamonds',
       'yellow gold, lacquer, tsavorite garnet, onyx',
       'yellow go

In [10]:
df_jewelry['color'] = df_jewelry['color'].str.replace('Gold','Yellow')

df_cartier['tags'] = df_cartier['tags'].str.replace('yellow gold','Yellow')
df_cartier['tags'] = df_cartier['tags'].str.replace('white gold','White')
df_cartier['tags'] = df_cartier['tags'].str.replace('pink gold','Rose')

In [11]:
df_jewelry.fillna(np.nan,inplace=True)
df_cartier.fillna(np.nan,inplace=True)

df_jewelry.replace('nan',np.nan,inplace=True)
df_cartier.replace('nan',np.nan,inplace=True)

### Determining price inflation with 10 random rings from the Cartier dataset

In [12]:
index_set = set()
while True:
    ind = rand.choice(df_cartier[df_cartier['categorie'] == 'rings'].index)
    index_set.add(ind)
    if len(index_set) == 10:
        break

index_list = list(index_set)

In [13]:
# Prices of 10 random cartier items from the dataset
start = [1110,20700,6000,2020,13600,1040,8800,1130,1380,171000]
# start = df_cartier.loc[index_list]['price'].values

# Prices on Cartier website. Taken in June 2021
end = [1220,22800,6600,2220,12800,1140,9650,1210,1520,192000]

dif = list(map(lambda l: (l[1] - l[0]) / l[0], zip(start,end)))

In [14]:
average_price_increase = sum(dif)/len(dif)
average_price_increase

0.0828532254367677

In [15]:
d_after = dt.datetime.strptime('062021', "%m%Y").date() # date of my Cartier webscrape
d_start = dt.datetime.strptime('042020', "%m%Y").date() # date of dataset Cartier webscrape
num_of_months = (d_after.year - d_start.year) * 12 + d_after.month - d_start.month

f'The number of months between my webscrape and the original webscrape: {num_of_months}'

'The number of months between my webscrape and the original webscrape: 14'

In [16]:
# Determining the monthly price inflation rate using the compound interest formula
P = 1 
n = 12 # months
A = average_price_increase + 1
t = num_of_months/12
r = round(n*((A/P)**(1/(t*n)) - 1),4)
print(f'The rate r = {r*100:.2f} % compounded monthly')
# This number will be used in the API for determining the current price

The rate r = 6.84 % compounded monthly


### Checking jewelry types and keeping only rings:

In [17]:
df_jewelry['jewel_type'].unique()

array(['Necklaces', 'Earrings', 'Bracelet', 'Ring', 'Cuff links',
       'Religious jewels', 'Cross', 'Jewels ID', 'Safety pins', nan,
       'Accessories', 'Link', 'Key ring', 'Pendant', 'Chain',
       'Wedding rings', 'Brooches'], dtype=object)

In [18]:
df_cartier['categorie'].unique()

array(['rings', 'bracelets', 'necklaces', 'earrings'], dtype=object)

In [19]:
df_ring_j = df_jewelry[df_jewelry['jewel_type'].isin(['Wedding rings','Ring'])]
df_ring_j.head()

,name,price,sex,jewel_type,brand,material,color,jewel_weight,rocks,rock_details,dimensions,chain_carat,chain_length,diameter,price_usd
7,Ring with emerald,1650,Women,Ring,Haritidis,Gold 18ct.,White,3.8,"Diamond, Emerald",Recrystallized Emerald 2.93ct & Diamond 0.56ct,NaN,NaN,NaN,NaN,1963.5
8,Monoch Ring,1200,Women,Ring,Haritidis,Gold 18ct.,Yellow,2.4,Diamond,Kent.Dress 0.11ct & Other Diamonds 0.26ct,NaN,NaN,NaN,NaN,1428.0
9,Ring halfway,1180,Women,Ring,Haritidis,Gold 18ct.,White,2.0,Diamond,Diamonds 0.45ct,NaN,NaN,NaN,NaN,1404.2
10,Ring halfway,980,Women,Ring,Haritidis,Gold 18ct.,White,2.3,Diamond,Diamonds 0.24ct,NaN,NaN,NaN,NaN,1166.2
11,Ring halfway,1850,Women,Ring,Haritidis,Gold 18ct.,White,2.8,Diamond,Diamonds 0.66ct,NaN,NaN,NaN,NaN,2201.5


In [20]:
df_ring_c = df_cartier[df_cartier['categorie'] == 'rings']
df_ring_c.head()

,categorie,title,price,tags,description,brand
0,rings,Love ring,1650.0,Yellow,"Love ring, 18K yellow gold. Width: 5.5mm.",Cartier
1,rings,Love ring,3650.0,platinum,"Love ring, 950/1000 platinum. Width: 5.5mm.",Cartier
2,rings,Love ring,1650.0,Rose,"Love ring, 18K rose gold. Width: 5.5mm.",Cartier
3,rings,Love ring,1770.0,White,"Love ring, 18K white gold. Width: 5.5mm.",Cartier
4,rings,Love wedding band,1070.0,Yellow,"Love wedding band, 18K yellow gold. Width: 3.6mm.",Cartier


In [21]:
df_ring_c.loc[df_ring_c['description'].str.contains('band',case=False,na=False),'jewel_type'] = 'Wedding rings'
df_ring_c.loc[df_ring_c['jewel_type'].isna(),'jewel_type'] = 'Ring'

df_ring_c.head()

,categorie,title,price,tags,description,brand,jewel_type
0,rings,Love ring,1650.0,Yellow,"Love ring, 18K yellow gold. Width: 5.5mm.",Cartier,Ring
1,rings,Love ring,3650.0,platinum,"Love ring, 950/1000 platinum. Width: 5.5mm.",Cartier,Ring
2,rings,Love ring,1650.0,Rose,"Love ring, 18K rose gold. Width: 5.5mm.",Cartier,Ring
3,rings,Love ring,1770.0,White,"Love ring, 18K white gold. Width: 5.5mm.",Cartier,Ring
4,rings,Love wedding band,1070.0,Yellow,"Love wedding band, 18K yellow gold. Width: 3.6mm.",Cartier,Wedding rings


## Feature Creation and Homogenization 
### Cartier Dataset

In [22]:
df_ring_c.loc[(df_ring_c['tags'].str.contains('White',na=False)) 
              & (df_ring_c['tags'].str.contains('Rose',na=False)) 
              & (df_ring_c['tags'].str.contains('Yellow',na=False)),'color'] = 'White & Yellow & Rose'

df_ring_c.loc[(df_ring_c['tags'].str.contains('White',na=False)) 
              & (df_ring_c['tags'].str.contains('Rose',na=False)),'color'] = 'White & Rose'

df_ring_c.loc[(df_ring_c['tags'].str.contains('White',na=False))
              & (df_ring_c['tags'].str.contains('Yellow',na=False)),'color'] = 'White & Yellow'

df_ring_c.loc[(df_ring_c['tags'].str.contains('Rose',na=False)) 
              & (df_ring_c['tags'].str.contains('Yellow',na=False)),'color'] = 'Yellow & Rose'

df_ring_c.loc[(df_ring_c['tags'].str.contains('White',na=False)),'color'] = 'White'

df_ring_c.loc[(df_ring_c['tags'].str.contains('Rose',na=False)),'color'] = 'Rose'

df_ring_c.loc[(df_ring_c['tags'].str.contains('Yellow',na=False)),'color'] = 'Yellow'

In [23]:
df_ring_c_gold = df_ring_c[df_ring_c['color'].isin(['Yellow', 'Rose', 'White']) 
                           & ~df_ring_c['description'].str.contains('‰ platinum')]

In [24]:
#df_ring_c_gold['width (mm)'] = df_ring_c_gold['description'].str.extractall(r"Width:(.+)mm").astype(float).droplevel('match')

In [25]:
c_diamond_carats = df_ring_c_gold['description'].str\
                    .extractall(r"diamonds totaling (\d+\.\d+) carat"
                               ,flags=re.IGNORECASE)\
                    .astype(float).droplevel('match')

total_c_diamond_carats = c_diamond_carats.groupby(c_diamond_carats.index).agg({0: sum})
df_ring_c_gold['total diamond carats'] = total_c_diamond_carats

In [26]:
gold_carats_df = df_ring_c_gold['description'].str\
                    .extractall(r"(\d+)K",
                               flags=re.IGNORECASE)\
                    .astype(int).droplevel('match')

final_gold_carats_df = gold_carats_df.groupby(gold_carats_df.index).agg({0: 'first'})
df_ring_c_gold['gold carats'] = final_gold_carats_df
df_ring_c_gold = df_ring_c_gold[~df_ring_c_gold['gold carats'].isna()]

In [27]:
c_jewel_weight = df_ring_c_gold['description'].str\
                    .extractall(r"totaling (\d+\.\d+) carat"
                                ,flags=re.IGNORECASE)\
                    .astype(float).droplevel('match')

total_c_jewel_weight = c_jewel_weight.groupby(c_jewel_weight.index).agg({0: sum})
df_ring_c_gold['jewel_weight'] = total_c_jewel_weight

In [28]:
df_ring_c_gold.head()

,categorie,title,price,tags,description,brand,jewel_type,color,total diamond carats,gold carats,jewel_weight
0,rings,Love ring,1650.0,Yellow,"Love ring, 18K yellow gold. Width: 5.5mm.",Cartier,Ring,Yellow,NaN,18.0,NaN
2,rings,Love ring,1650.0,Rose,"Love ring, 18K rose gold. Width: 5.5mm.",Cartier,Ring,Rose,NaN,18.0,NaN
3,rings,Love ring,1770.0,White,"Love ring, 18K white gold. Width: 5.5mm.",Cartier,Ring,White,NaN,18.0,NaN
4,rings,Love wedding band,1070.0,Yellow,"Love wedding band, 18K yellow gold. Width: 3.6mm.",Cartier,Wedding rings,Yellow,NaN,18.0,NaN
5,rings,Love wedding band,1070.0,Rose,"Love wedding band, 18K rose gold. Width: 3.6mm.",Cartier,Wedding rings,Rose,NaN,18.0,NaN


In [29]:
first = df_ring_c_gold['description'].str.split(',', n=1, expand=True)[1]
second = first.str.split('gold.', n=1, expand=True)[1]
df_ring_c_gold['rock_details'] = second.str.split('Width', n=1, expand=True)[0]

In [30]:
bad_set_c = ['wide from size']

bad_dict_c = {'brilliant-diamonds wide also available as midi ring':'brilliant-diamonds',
              'brilliant-diamonds all-gold ring paved ring':'brilliant-diamonds',
              'black-ceramic tto ring ceramic ring':'black-ceramic'}

def replace_func_c(x):
    val = x
    for num in list(map(str,list(range(10)))):
         val = val.replace(num,'')
    val = val.lower()
    
    val = val.replace('.','')
    val = val.replace(',','')
    val = val.replace('totaling','')
    val = val.replace('carats','')
    val = val.replace('carat','')
    val = val.replace('set with','')
    val = val.replace('-cut','')
    val = val.replace('mm','')
    val = val.replace('width','')
    val = val.replace(':','')
    val = val.replace(' a ',' ')
    val = val.replace(' to ',' ')
    val = val.replace(' and ',' ')
    val = val.replace('and ',' ')
    val = val.replace(' an ',' ')
    val = val.replace('following','')
    val = val.replace('metrics','')
    val = val.replace('k white gold','')
    val = val.replace('k rose gold','')
    val = val.replace('k pink gold','')
    val = val.replace('k yellow gold','')
    val = val.replace('white gold','')
    val = val.replace('rose gold','')
    val = val.replace('sizes','')
    val = val.replace('all-gold ring paved ring','')
    val = val.replace('beads','')
    val = val.replace('eyes','')
    val = val.replace('nose','')
    val = val.replace('with','')
    val = val.replace(' of','')
    val = val.replace(' center stone ',' ') 
    val = val.replace(' tto ring ceramic ring','')
              
    val = val.replace('black ','black-')
    val = val.replace('rose ','rose-')
    val = val.replace('blue ','blue-')
    val = val.replace('yellow ','yellow-')
    val = val.replace('green ','green-')
    val = val.replace('orange ','orange-')
    val = val.replace('brilliant ','brilliant-')
    val = val.replace('tsavorite ','tsavorite-')
    val = val.replace('pear-shaped ','pear-shaped-') 
    val = val.replace('baguette ','baguette-')
    val = val.replace('princess ','princess-')
    val = val.replace('troidia ','troidia-')
    val = val.replace('brilliant-pavé ','brilliant-pavé-')
    val = val.replace('mother-of pearl','mother-of-pearl')
    val = val.replace('gray mother-of-pearl','gray-mother-of-pearl')

    val = val.replace('(','')
    val = val.replace(')','')
    
    val = val.replace('    ',' ')
    val = val.replace('   ',' ')
    val = val.replace('  ',' ')
    
    val = val.strip()
    
#     if val in ('oval',):
#         print(val,x)
    
    if val in bad_set_c:
        return ''
    elif val in bad_dict_c:
        return bad_dict_c[val]
#     elif val == 'nan':
#         return np.nan
#     else:
#         return val
    return val

In [31]:
df_ring_c_gold['abbrev description'] = df_ring_c_gold['rock_details'].apply(lambda x: replace_func_c(str(x)))

In [32]:
df_ring_c_gold['rock_details'].apply(lambda x: replace_func_c(str(x))).unique()

array(['', 'brilliant-diamonds', 'brilliant-diamond',
       'black-ceramic brilliant-diamonds', 'rose-sapphire',
       'rose-sapphire blue-sapphire yellow-sapphire green-garnet orange-garnet amethyst',
       'brilliant-diamonds emeralds onyx',
       'onyx black-lacquer tsavorite-garnets', 'onyx tsavorite-garnets',
       'tsavorite-garnets onyx',
       'black-lacquer brilliant-diamonds tsavorite-garnet',
       'brilliant-diamonds tsavorite-garnets onyx black-lacquer',
       'onyx emeralds brilliant-diamonds',
       'black-lacquer tsavorite-garnets onyx',
       'brilliant-diamonds tsavorite-garnets onyx',
       'black-lacquer peridots onyx', 'diamonds emeralds onyx',
       'gray-mother-of-pearl emeralds sapphire brilliant-diamonds',
       'gray-mother-of-pearl emerald sapphire brilliant-diamonds',
       'emeralds brilliant-diamonds', 'emeralds onyx brilliant-diamonds',
       'spinels brilliant-diamonds',
       'emeralds carnelians brilliant-diamond',
       'chrysoprases 

#### Putting a value of 1 in every column that corresponds to a term in the ring description.

In [33]:
for idx in df_ring_c_gold['abbrev description'].index:
    description = df_ring_c_gold['abbrev description'].loc[idx]
    if description is np.nan:
        continue
    for term in description.split():
        df_ring_c_gold.loc[idx,term] = 1

### Jewelry Dataset

In [34]:
# Only Gold rings have sufficient data
df_ring_j_gold = df_ring_j[df_ring_j['material'].isin(['Gold 18ct.', 'Gold 14ct.', 'Gold 9ct.'])]

In [35]:
j_diamond_carats = df_ring_j_gold['rock_details'].str\
                    .extractall(r"diamond.? (?:triangle |brilliant |baguette |princess )?(\d+\.\d+)(?!\.|mm| mm|-)"
                                ,flags=re.IGNORECASE)\
                    .astype(float).droplevel('match')

total_j_diamond_carats = j_diamond_carats.groupby(j_diamond_carats.index).agg({0: sum})
df_ring_j_gold['total diamond carats'] = total_j_diamond_carats

In [36]:
df_ring_j_gold['gold carats'] = df_ring_j_gold['material'].str\
                                    .extractall(r"(\d+)(?:ct)?",
                                                flags=re.IGNORECASE)\
                                    .astype(float).droplevel('match')

In [37]:
df_ring_j_gold.head()

,name,price,sex,jewel_type,brand,material,color,jewel_weight,rocks,rock_details,dimensions,chain_carat,chain_length,diameter,price_usd,total diamond carats,gold carats
7,Ring with emerald,1650,Women,Ring,Haritidis,Gold 18ct.,White,3.8,"Diamond, Emerald",Recrystallized Emerald 2.93ct & Diamond 0.56ct,NaN,NaN,NaN,NaN,1963.5,0.56,18.0
8,Monoch Ring,1200,Women,Ring,Haritidis,Gold 18ct.,Yellow,2.4,Diamond,Kent.Dress 0.11ct & Other Diamonds 0.26ct,NaN,NaN,NaN,NaN,1428.0,0.26,18.0
9,Ring halfway,1180,Women,Ring,Haritidis,Gold 18ct.,White,2.0,Diamond,Diamonds 0.45ct,NaN,NaN,NaN,NaN,1404.2,0.45,18.0
10,Ring halfway,980,Women,Ring,Haritidis,Gold 18ct.,White,2.3,Diamond,Diamonds 0.24ct,NaN,NaN,NaN,NaN,1166.2,0.24,18.0
11,Ring halfway,1850,Women,Ring,Haritidis,Gold 18ct.,White,2.8,Diamond,Diamonds 0.66ct,NaN,NaN,NaN,NaN,2201.5,0.66,18.0


In [38]:
bad_dict_j = {'diamond diamonds perimeter': 'diamond perimeter-diamonds', 
              'file krishzafori sapphires':'sapphires', 'brilliant diamonds': 'brilliant-diamonds',
              'diamonds rest': 'diamonds', 'dimensions stone': 'delete', 'topaz violac': 'violac-topaz',
              'diamond diamonds princess': 'diamond princess-diamonds', 'sinusite': 'delete',
              'baguette brilliant diamonds': 'baguette-brilliant-diamonds',
              'brilliant diamonds marquise': 'marquise-brilliant-diamonds',  
              'baguette brilliant sapphires': 'baguette-brilliant-sapphires', 
              'baguette brilliant emerald':'baguette-brilliant-emerald',
              'diamond diamonds oval':'oval-diamond diamonds',
              'baguette brilliant diamond diamonds': 'baguette-diamond brilliant-diamonds'}

def replace_func_j(x):
    val = x
#     for num in list(map(str,list(range(10)))):
#          val = val.replace(num,'')
    val = re.sub('\d+\.?\d*\.?(ct|mm|t|pm)','',val)
    val = re.sub('\d+.\d+','',val)
    val = val.lower()
    
    val = re.sub('\d+\.?\d*\.?(ct|mm|t|pm)','',val)
    val = re.sub('(?<!vs)\d','',val)
    
    val = val.replace('-','')
    val = val.replace('centrally ','')
    val = val.replace('central ','')
    val = val.replace('f.w.p','fresh-water-pearl')
    val = val.replace('kent','')
    val = val.replace(' x','')
    val = val.replace('diam.','diamond')
    val = val.replace('diamonda','diamonds')
    val = val.replace('&','')
    val = val.replace('and','')
    val = val.replace('kashmir','kashmir-')
    val = val.replace('spinell','spinel')
    val = val.replace('topozy','topaz')
    val = val.replace('zisite','zoisite')
    val = val.replace('sitrin','citrine')
    val = val.replace('moronganic','morganite')
    val = val.replace('mop','mother-of-pearl')
    val = val.replace('mofpearl','mother-of-pearl')
    val = val.replace('akoumara','aquamarine')
    val = val.replace('coffee ','coffee-')
    val = val.replace('black ','black-')
    val = val.replace('baby ','baby-')
    val = val.replace('white ','white-')
    val = val.replace('whites ','white-')
    val = val.replace('green ','green-')
    val = val.replace('black ','black-')
    val = val.replace('pearl akoya','akoya-pearl')
    val = val.replace('aqua ','aqua-')
    val = val.replace('blue ','blue-')
    val = val.replace('red ','red-')
    val = val.replace('pearl edison','edison-pearl')
    val = val.replace('blazing ','blazing-')
    val = val.replace('paraiba ','paraiba-')
    val = val.replace('london ','london-')
    val = val.replace('triangle ','triangle-')
    val = val.replace('pink ','pink-')
    val = val.replace('rainforest ','rainforest-')
    val = val.replace('pearl south sea','south-sea-pearl')
    val = val.replace('recrystallized ','recrystallized-')
    val = val.replace('fresh water pearl','fresh-water-pearl')
    val = val.replace('dress','')
    
    val = val.replace('diamonds white-','white-diamonds')
    val = val.replace('diamonds coffee-','coffee-diamonds')

    val = val.replace('gronada','')
    val = val.replace('zer','')
    val = val.replace('day','')

    val = val.replace('(','')
    val = val.replace(')','')

    val = val.replace('.mm','')
    val = val.replace(' mm','')
    val = val.replace('kent.dress','')

    val = val.replace('.ct','')
    val = val.replace(' ct','')
    val = val.replace(' other ',' ')
    val = val.replace(' rest',' ')
    
    val = val.replace(' .. ',' ')
    val = val.replace(' . ',' ')
    val = val.replace('.','')
    val = val.replace('   ',' ')
    val = val.replace('  ',' ')
    
    val = val.strip()
    
    val = val.replace('pearl pm','pearl')
    val = val.replace('pearl fresh-water-pearl','fresh-water-pearl')
    
    val = val.replace('- ','-')
    val = val.replace(' -','-')
    val = val.replace('/',' ')
    
    val = ' '.join(sorted(list(set(val.split()))))

    if val in bad_dict_j:
        return bad_dict_j[val]
    elif val == 'nan':
        return np.nan
    else:
        return val

In [39]:
df_ring_j_gold['abbrev description'] = df_ring_j_gold['rock_details'].apply(lambda x: replace_func_j(str(x)))

In [40]:
df_ring_j_gold = df_ring_j_gold[df_ring_j_gold['abbrev description'] != 'delete']

In [41]:
df_ring_j_gold['abbrev description'][~df_ring_j_gold['abbrev description'].isna()].unique()

array(['diamond recrystallized-emerald', 'diamonds', 'diamond',
       'diamond topaz', 'diamond emerald', 'diamonds sapphire',
       'diamond white-topaz', 'diamond sapphire', 'diamond tanzanite',
       'diamond london-blue-topaz', 'diamond morganite', 'pearl',
       'diamonds ruby', 'diamond white-sapphire', 'blue-topaz diamonds',
       'diamonds emerald', 'diamond ruby', 'blue-topaz diamond',
       'citrine diamond', 'blue-sapphire diamonds', 'emerald',
       'blue-sapphire diamond', 'diamond diamonds f vs2',
       'amethyst diamond', 'topaz', 'diamonds qendrad', 'diamond e vs2',
       'diamond f vs2', 'diamond g vs2', 'diamond e vvs1',
       'diamond triangle-diamonds', 'diamonds spinel',
       'diamonds emeralds', 'blue-pink-sapphires diamonds',
       'blue-sapphires diamonds', 'diamonds tanzanite',
       'diamond princess-diamonds', 'diamond diamonds emerald',
       'black-diamond white-diamonds', 'diamond pink-sapphire',
       'diamond opal', 'aquamarine diamonds',

#### Putting a value of 1 in every column that corresponds to a term in the ring description.

In [42]:
for idx in df_ring_j_gold['abbrev description'].index:
    description = df_ring_j_gold['abbrev description'].loc[idx]
    if description is np.nan:
        continue
    for term in description.split():
        df_ring_j_gold.loc[idx,term] = 1

## Combining datasets

In [43]:
print(list(df_ring_c_gold.columns))

['categorie', 'title', 'price', 'tags', 'description', 'brand', 'jewel_type', 'color', 'total diamond carats', 'gold carats', 'jewel_weight', 'rock_details', 'abbrev description', 'brilliant-diamonds', 'brilliant-diamond', 'black-ceramic', 'rose-sapphire', 'blue-sapphire', 'yellow-sapphire', 'green-garnet', 'orange-garnet', 'amethyst', 'emeralds', 'onyx', 'black-lacquer', 'tsavorite-garnets', 'tsavorite-garnet', 'peridots', 'diamonds', 'gray-mother-of-pearl', 'sapphire', 'emerald', 'spinels', 'carnelians', 'chrysoprases', 'lapis', 'lazulis', 'diamond', 'princess-diamond', 'brilliant-pavé-diamonds', 'troidia-diamond', 'baguette-diamonds', 'princess-diamonds', 'troidia-diamonds', 'rhodium-finish', 'pear-shaped-diamonds', 'amazonite', 'coral']


In [44]:
c_cols = df_ring_c_gold.drop(['categorie', 'title','tags', 'description','rock_details', 'abbrev description',
                              'abbrev description'],axis=1).columns

cartier_final = df_ring_c_gold[c_cols]

In [45]:
print(list(df_ring_j_gold.columns))

['name', 'price', 'sex', 'jewel_type', 'brand', 'material', 'color', 'jewel_weight', 'rocks', 'rock_details', 'dimensions', 'chain_carat', 'chain_length', 'diameter', 'price_usd', 'total diamond carats', 'gold carats', 'abbrev description', 'diamond', 'recrystallized-emerald', 'diamonds', 'topaz', 'emerald', 'sapphire', 'white-topaz', 'tanzanite', 'london-blue-topaz', 'morganite', 'pearl', 'ruby', 'white-sapphire', 'blue-topaz', 'citrine', 'blue-sapphire', 'f', 'vs2', 'amethyst', 'qendrad', 'e', 'g', 'vvs1', 'triangle-diamonds', 'spinel', 'emeralds', 'blue-pink-sapphires', 'blue-sapphires', 'princess-diamonds', 'black-diamond', 'white-diamonds', 'pink-sapphire', 'opal', 'aquamarine', 'sapphires', 'baguette-brilliant-diamonds', 'green-sapphire', 'marquise-brilliant-diamonds', 'rodolite', 'coffee-diamonds', 'd', 'vvs2', 'akoya-pearl', 'oval-diamond', 'south-sea-pearl', 'baguette-brilliant-sapphires', 'baguette-brilliant-emerald', 'brilliant-diamonds', 'baguette-diamond', 'si', 'vs1', 'bl

In [46]:
j_cols = df_ring_j_gold.drop(['price','name','sex','material','rocks', 'rock_details', 'dimensions', 'chain_carat',
                              'chain_length','diameter','abbrev description'],axis=1).columns

jewelry_final = df_ring_j_gold[j_cols]

In [47]:
c_cols

Index(['price', 'brand', 'jewel_type', 'color', 'total diamond carats',
       'gold carats', 'jewel_weight', 'brilliant-diamonds',
       'brilliant-diamond', 'black-ceramic', 'rose-sapphire', 'blue-sapphire',
       'yellow-sapphire', 'green-garnet', 'orange-garnet', 'amethyst',
       'emeralds', 'onyx', 'black-lacquer', 'tsavorite-garnets',
       'tsavorite-garnet', 'peridots', 'diamonds', 'gray-mother-of-pearl',
       'sapphire', 'emerald', 'spinels', 'carnelians', 'chrysoprases', 'lapis',
       'lazulis', 'diamond', 'princess-diamond', 'brilliant-pavé-diamonds',
       'troidia-diamond', 'baguette-diamonds', 'princess-diamonds',
       'troidia-diamonds', 'rhodium-finish', 'pear-shaped-diamonds',
       'amazonite', 'coral'],
      dtype='object')

In [48]:
j_cols

Index(['jewel_type', 'brand', 'color', 'jewel_weight', 'price_usd',
       'total diamond carats', 'gold carats', 'diamond',
       'recrystallized-emerald', 'diamonds', 'topaz', 'emerald', 'sapphire',
       'white-topaz', 'tanzanite', 'london-blue-topaz', 'morganite', 'pearl',
       'ruby', 'white-sapphire', 'blue-topaz', 'citrine', 'blue-sapphire', 'f',
       'vs2', 'amethyst', 'qendrad', 'e', 'g', 'vvs1', 'triangle-diamonds',
       'spinel', 'emeralds', 'blue-pink-sapphires', 'blue-sapphires',
       'princess-diamonds', 'black-diamond', 'white-diamonds', 'pink-sapphire',
       'opal', 'aquamarine', 'sapphires', 'baguette-brilliant-diamonds',
       'green-sapphire', 'marquise-brilliant-diamonds', 'rodolite',
       'coffee-diamonds', 'd', 'vvs2', 'akoya-pearl', 'oval-diamond',
       'south-sea-pearl', 'baguette-brilliant-sapphires',
       'baguette-brilliant-emerald', 'brilliant-diamonds', 'baguette-diamond',
       'si', 'vs1', 'black-diamonds', 'black-sapphire',
       'ed

In [49]:
jewelry_final.rename(columns={'price_usd':'price'}, inplace=True)
jewelry_final['brand'] = jewelry_final['brand'].fillna('')

In [50]:
final_df = pd.concat([cartier_final,jewelry_final], ignore_index=True)

In [51]:
final_df.fillna(0, inplace=True)

In [52]:
print(final_df.isna().sum().sum())

0


In [53]:
print(list(final_df.columns))

['price', 'brand', 'jewel_type', 'color', 'total diamond carats', 'gold carats', 'jewel_weight', 'brilliant-diamonds', 'brilliant-diamond', 'black-ceramic', 'rose-sapphire', 'blue-sapphire', 'yellow-sapphire', 'green-garnet', 'orange-garnet', 'amethyst', 'emeralds', 'onyx', 'black-lacquer', 'tsavorite-garnets', 'tsavorite-garnet', 'peridots', 'diamonds', 'gray-mother-of-pearl', 'sapphire', 'emerald', 'spinels', 'carnelians', 'chrysoprases', 'lapis', 'lazulis', 'diamond', 'princess-diamond', 'brilliant-pavé-diamonds', 'troidia-diamond', 'baguette-diamonds', 'princess-diamonds', 'troidia-diamonds', 'rhodium-finish', 'pear-shaped-diamonds', 'amazonite', 'coral', 'recrystallized-emerald', 'topaz', 'white-topaz', 'tanzanite', 'london-blue-topaz', 'morganite', 'pearl', 'ruby', 'white-sapphire', 'blue-topaz', 'citrine', 'f', 'vs2', 'qendrad', 'e', 'g', 'vvs1', 'triangle-diamonds', 'spinel', 'blue-pink-sapphires', 'blue-sapphires', 'black-diamond', 'white-diamonds', 'pink-sapphire', 'opal', 'a

In [54]:
final_df.head()

,price,brand,jewel_type,color,total diamond carats,gold carats,jewel_weight,brilliant-diamonds,brilliant-diamond,black-ceramic,...,pink-sapphires,perimeter-diamonds,zoisite,blue-diamond,red-topaz,pink-quartz,quartz,tourmaline,i,mother-of-pearl
0,1650.0,Cartier,Ring,Yellow,0.0,18.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1650.0,Cartier,Ring,Rose,0.0,18.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1770.0,Cartier,Ring,White,0.0,18.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1070.0,Cartier,Wedding rings,Yellow,0.0,18.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1070.0,Cartier,Wedding rings,Rose,0.0,18.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [55]:
final_df = final_df[final_df['brand'].isin(['Haritidis', 'Cartier'])]

In [56]:
final_df[['price', 'brand', 'jewel_type', 'color', 'total diamond carats', 'gold carats', 'jewel_weight']].describe()

,price,total diamond carats,gold carats,jewel_weight
count,1966.000000,1966.000000,1966.000000,1966.000000
mean,2191.866109,0.212391,16.599695,2.819672
std,6417.237121,0.444998,1.914450,1.865009
min,77.350000,0.000000,9.000000,0.000000
25%,368.900000,0.000000,14.000000,1.700000
50%,873.460000,0.100000,18.000000,2.500000
75%,1676.500000,0.270000,18.000000,3.600000
max,85500.000000,7.160000,18.000000,15.000000


In [57]:
final_df['gold carats'].unique()

array([18., 14.,  9.])

In [58]:
final_df.groupby('brand')['price'].mean()

brand
Cartier      11771.462264
Haritidis     1034.012982
Name: price, dtype: float64

In [59]:
final_df.groupby('brand')['price'].count()

brand
Cartier       212
Haritidis    1754
Name: price, dtype: int64

### Dummy Variables

In [60]:
final_df = pd.get_dummies(final_df, drop_first=True)
final_df.head()

,price,total diamond carats,gold carats,jewel_weight,brilliant-diamonds,brilliant-diamond,black-ceramic,rose-sapphire,blue-sapphire,yellow-sapphire,...,i,mother-of-pearl,brand_Haritidis,jewel_type_Wedding rings,color_Rose,color_White,color_White & Rose,color_White & Yellow,"color_White, Yellow & Rose",color_Yellow
0,1650.0,0.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,0,0,0,0,0,0,1
1,1650.0,0.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,0,1,0,0,0,0,0
2,1770.0,0.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,0,0,1,0,0,0,0
3,1070.0,0.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,1,0,0,0,0,0,1
4,1070.0,0.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,1,1,0,0,0,0,0


## Model Creation

In [61]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

def scorers(target,pred):
    print('MSE:', mean_squared_error(target,pred))
    print('MAE:', mean_absolute_error(target,pred))
    print('R^2:', r2_score(target,pred))

### Oversampling for models

In [62]:
oversampling_count = final_df.groupby('brand_Haritidis')['price'].count()[1]

In [63]:
underrep_idxs = final_df[final_df['brand_Haritidis'] == 0].index

In [64]:
idx_list = []
for i in range(oversampling_count):
    idx_list.append(rand.choice(underrep_idxs))

In [65]:
cartier_rows = final_df.loc[idx_list]
haritidis_rows = final_df[final_df['brand_Haritidis'] == 1] 

In [66]:
oversampled_df = pd.concat([cartier_rows,haritidis_rows], ignore_index=True)

In [67]:
y = oversampled_df['price']
X = oversampled_df.drop('price',axis=1)

In [68]:
stratify_by_field_1 = oversampled_df['brand_Haritidis']
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=stratify_by_field_1, 
                                                    test_size=0.20, random_state = 0)

### Previously trained models with cross validation:

In [69]:
# clf_ridge = Ridge(random_state=0)

In [70]:
# clf_rfr = RandomForestRegressor(random_state=0)

In [71]:
# from sklearn.model_selection import GridSearchCV

In [72]:
# alphas = np.array([1,0.1,0.01,0.001,0.0001,0])

# grid_ridge = GridSearchCV(estimator=clf_ridge, param_grid=dict(alpha=alphas),
#                           verbose=2, n_jobs=-1)

In [73]:
# params = {'bootstrap': [True, False],
#          'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
#          'max_features': ['auto', 'sqrt'],
#          'min_samples_leaf': [1, 2, 4],
#          'min_samples_split': [2, 5, 10],
#          'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}

# grid_rfr = GridSearchCV(estimator=clf_rfr, param_grid=params,
#                         verbose=2, n_jobs=-1)

In [74]:
# grid_ridge.fit(X_train, y_train);

In [75]:
# print(grid_ridge.best_score_)
# print(grid_ridge.best_estimator_)

In [76]:
# grid_rfr.fit(X_train, y_train);

In [77]:
# print(grid_rfr.best_score_)
# print(grid_rfr.best_estimator_)

In [78]:
# y_pred_ridge = grid_ridge.predict(X_test)

In [79]:
# y_pred_rfr = grid_rfr.predict(X_test)

### Results from previous training:

#### Ridge:
##### R^2
0.8565809888502182 
<br>
##### Best parameters
Ridge(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=0, solver='auto', tol=0.001)

#### Random Forest:
##### R^2
0.9889098573710595
<br>
##### Best parameters
RandomForestRegressor(bootstrap=False, criterion='mse', max_depth=30,
                      max_features='sqrt', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=5,
                      min_weight_fraction_leaf=0.0, n_estimators=200,
                      n_jobs=None, oob_score=False, random_state=0, verbose=0,
                      warm_start=False)

In [80]:
clf_ridge = Ridge(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=None, normalize=False, random_state=0,
                  solver='auto', tol=0.001)

In [81]:
clf_rfr = RandomForestRegressor(bootstrap=False, criterion='mse', max_depth=30, max_features='sqrt', max_leaf_nodes=None,
                                min_impurity_decrease=0.0, min_samples_leaf=1, min_samples_split=5,
                                min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=None, oob_score=False, random_state=0,
                                verbose=0, warm_start=False)

In [82]:
clf_ridge.fit(X_train, y_train);

In [83]:
clf_rfr.fit(X_train, y_train);

In [84]:
y_pred_ridge = clf_ridge.predict(X_test)

In [85]:
y_pred_rfr = clf_rfr.predict(X_test)

### Results

In [86]:
scorers(y_test, y_pred_ridge)

MSE: 24432124.27507925
MAE: 2315.630810856286
R^2: 0.864809964691384


In [87]:
scorers(y_test, y_pred_rfr)

MSE: 1325334.1967531599
MAE: 348.23185693560146
R^2: 0.992666541196439


### Exporting Model

In [88]:
import joblib

In [89]:
joblib.dump(clf_ridge, 'ridge_model.pkl', compress=9)
joblib.dump(clf_rfr, 'rfr_model.pkl', compress=9);